<a href="https://colab.research.google.com/github/AryanChaks/Llama2-FineTune/blob/main/Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb #for training parametre detection

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/

In [ ]:
from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

dataset_name = 'AryanChaks/Grouped_Open_Orca'
dataset = load_dataset(dataset_name,split="train")

In [ ]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 5005
})


In [ ]:
print(dataset[0])

{'text': '### Human: {\'id\': \'cot.86217\', \'system_prompt\': \'You are an AI assistant that helps people find information.\', \'question\': "James runs a TV show and there are 5 main characters and 4 minor characters. He pays the minor characters $15, 000 each episode. He paid the major characters three times as much. How much does he pay per episode? Let\'s be accurate as possible."\n### Assistant: \'response\': \'James pays the minor characters $15, 000 each episode. Since there are 4 minor characters, he pays them a total of 4 * $15, 000 = $60, 000 per episode.\\n\\nThe major characters are paid three times as much. So, each major character gets paid 3 * $15, 000 = $45, 000 per episode.\\n\\nThere are 5 main characters, so he pays them a total of 5 * $45, 000 = $225, 000 per episode.\\n\\nIn total, James pays $225, 000 (major characters) + $60, 000 (minor characters) = $285, 000 per episode.\'}'}


In [ ]:
formatted_examples = []

# Iterate through the original dataset and convert each example to the specified format
for example in dataset:
    example_dict = eval(example['text'])  # Convert string representation to dictionary
    input_text = f"### Human: {example_dict['system_prompt']} {example_dict['question']}"
    response = f"### Assistant: {example_dict['response']}"
    formatted_example = {"text": f"{input_text}{response}"}
    formatted_examples.append(formatted_example)

# Print the format of the first line in the formatted_examples list
print(formatted_examples[0])

SyntaxError: invalid syntax (<string>, line 2)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

SyntaxError: unterminated string literal (detected at line 3) (<ipython-input-37-954d0624670b>, line 3)

In [ ]:
def formatting_func(example):
    # Assuming 'text' field is a string containing both question and answer separated by "### Assistant:"
    parts = example['text'].split("### Assistant:")
    human_part = parts[0].strip()
    assistant_part = parts[1].strip() if len(parts) > 1 else ''
    return {
        'input_text': human_part,  # Everything before "### Assistant:"
        'target_text': assistant_part  # Everything after "### Assistant:"
    }


# Update your SFTTrainer instantiation with formatting_func
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_examples,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,  # As per your setup
    formatting_func=formatting_func  # Add this line
)

AttributeError: 'list' object has no attribute 'column_names'

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 161.06 MiB is free. Process 10378 has 14.59 GiB memory in use. Of the allocated memory 14.17 GiB is allocated by PyTorch, and 295.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF